### 07. AI-agent

- En AI-agent är ett system som kan agera autonomt för att uppnå ett mål
    - ta in information (input),
    - fatta beslut baserat på regler, logik eller maskininlärning,
    - och utföra åtgärder (t.ex. kalla på verktyg eller API:er).

https://python.langchain.com/docs/integrations/chat/google_generative_ai/

![07 diagram](docs/07.drawio.png)

In [10]:
import os
import json

from dotenv import load_dotenv
import httpx

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool #decorator
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent


load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")  # load the api key from another file (.env)

In [11]:
@tool
def get_location_coordinates(location: str) -> dict:
    """returns coordinates in latitude and longitude for the given location

    Args:
        location (str): the location

    Returns:
        dict: latitude and longitude
    """

    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={API_KEY}"

    response = httpx.get(url)
    result = json.loads(response.text)
    location = result["results"][0]["geometry"]["location"]

    return {"latitude": location["lat"], "longitude": location["lng"]}

In [12]:
@tool
def get_current_temperature(latitude: str, longitude: str) -> dict:
    """return the current temperatire for the given coordinates

    Args:
        latitude (str): latitude
        longitude (str): longitude

    Returns:
        dict: the temperature
    """

    url = f"https://weather.googleapis.com/v1/currentConditions:lookup?key={API_KEY}&location.latitude={latitude}&location.longitude={longitude}"
    response = httpx.get(url)
    result = json.loads(response.text)

    return {
        "temperature": result["temperature"]["degrees"],
        "unit": result["temperature"]["unit"],
    }

In [13]:
memory = MemorySaver()
client = ChatGoogleGenerativeAI(model="gemini-2.0-flash", api_key=API_KEY)
tools = [get_location_coordinates, get_current_temperature]

agent_executor = create_react_agent(client, tools, checkpointer=memory)

In [17]:
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="what is the current temperature in moheda?")]},
    config=config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

what is the current temperature in moheda?
================================== Ai Message ==================================
Tool Calls:
  get_location_coordinates (51bb9cbc-e46c-43f5-8314-3a7f95c44c86)
 Call ID: 51bb9cbc-e46c-43f5-8314-3a7f95c44c86
  Args:
    location: moheda
================================= Tool Message =================================
Name: get_location_coordinates

{"latitude": 57.0027566, "longitude": 14.5732973}
================================== Ai Message ==================================
Tool Calls:
  get_current_temperature (1d8e5666-c67d-47dd-aca0-a34470a731f4)
 Call ID: 1d8e5666-c67d-47dd-aca0-a34470a731f4
  Args:
    longitude: 14.5732973
    latitude: 57.0027566
================================= Tool Message =================================
Name: get_current_temperature

{"temperature": 19.1, "unit": "CELSIUS"}
================================== Ai Message ==============